# VII. Signing with CLSAG

Alice has created all of the data Bob needs to receive Monero. Now Alice will create a signature proving she owns the Monero she is sending to Bob while at the same time hiding her true owned output among 15 other outputs she doesn't own.

There is one primary goal for this section:
1. Generate a valid CLSAG signature proving ownership of Monero Alice is spending

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 6](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)

**It is highly recommended that the reader read chapter 6 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. calculating the commitment to zero private key
2. identifying ring member data
3. recreating ring member commitment for mining output decoy
4. creating the message to be signed
5. obtaining CLSAG input data
6. creating a valid CLSAG signature

### Summary (Introduction)
**What we start with:**
* spend key image: `5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01`
* spend private key: `9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06`
* spend pseudo-commitment: `57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3`
* received input blinding factor: `67f2cd5597666bef76bf1dd4a40d201ea4b4013848332b68bbabcbbe8bf9a408`
* pseudo-commitment blinding factor: `3d5a7e3341949f499dde51deeac45cbf17b8d2ad073c7a28e0a0134ed46eaa08`

**What we end up with:**
* commitment to zero private key
* ring members
* CLSAG random number inputs (s)
* CLSAG verification input (c1)
* CLSAG commitment to zero key image (D)

## 1. Calculating the commitment to zero private key

Alice received Monero in a commitment in transaction be71, and is spending all of that Monero in a new transaction that has commitments for Bob and for zero change sent to a random address. A validator needs to ensure the inputs amounts equal the output amounts. If Alice directly references the prior output then the validator would be able to determine which output Alice owned and is spending, which would defeat Monero's privacy. Instead the process looks more like this:

1. Validate that the sum of output commitments (including transaction fees) equal the sum of pseduo-commitments
2. Validate that the difference in amounts between the real prior commitment and the pseduo-commitment is zero

To perform step 2 Alice has to create a commitment to zero that proves the amount in the pseduo-commitment is equal to the amount in the real prior commitment. In Alice's example transaction there is a single prior commitment and a single pseduo-commitment, and the only difference between the two is the blinding factors used (y). Alice will use the difference in blinding factors as a commitment to zero.

The output Alice received was generated with `y_received*G + bH`; and the pseduo-commitment Alice created was generated with `y_pseudo*G + bH`. Subtracting the second equation from the first leaves `(y_pseudo - y_received)*G`. The private key for that difference is `(y_pseudo - y_received)`. That will be the commitment to zero private key. 

In [1]:
'''
Zero to Monero: Second Edition; Section 5.5
'''


'''
To create the commitment to zero this code performs the following:

  1. Subtracts the old blinding factor from new blinding factor
'''

import nacl.bindings

from binascii import hexlify, unhexlify


# define a function to subtract ed25519 scalars
scalar_sub = nacl.bindings.crypto_core_ed25519_scalar_sub


# blinding factor from prior output Alice received (y_prior_input)
y_prior_input_commitment = "67f2cd5597666bef76bf1dd4a40d201ea4b4013848332b68bbabcbbe8bf9a408"

# blinding factor from pseudo-commitment Alice created (y_pseudo_comitment)
y_pseudo_commitment = "3d5a7e3341949f499dde51deeac45cbf17b8d2ad073c7a28e0a0134ed46eaa08"

# difference in blinding factors (y_prior_input - y_pseudo_commitment)
commitment_to_zero_private_key = scalar_sub(unhexlify(y_prior_input_commitment), unhexlify(y_pseudo_commitment))


print(f"Commitment to zero private key: {hexlify(commitment_to_zero_private_key).decode()}")

Commitment to zero private key: 176c457f7035defdaf7dc3989842a2738cfc2e8a40f7b03fdb0ab870b78afa0f


## 2. Identifying ring member data

Monero currently (v18) uses 15 decoy ring members for each output spent. The decoys are selected by wallet software and are visible in the transaction data. The decoys Alice uses when she sends to Bob will be extracted from actual transaction data for this worked example.

The ring members represent output indexes. Remember that a block can contain multiple transactions, and a transaction can contain multiple outputs. This means that to retrieve the neccessary data for a given transaction output, it is a requirement to know a cumulative count of prior outputs to determine which block and transaction contain that specific output. That information is stored in the blockchain, but for this worked example an external tool https://xmrchain.net will be used.   

In [2]:
'''
Zero to Monero: Second Edition; Appendix A
'''


'''
To identify ring member output index offsets this code performs the following:

  1. Reads the key_offsets field directly from the actual spend Alice -> Bob
'''

import json

# read data directly from the saved transaction (Alice -> Bob)
with open("../../transactions/outbound/txn_9e29.json", "r") as file:
    txn = json.load(file)


# ring members selected by wallet software when Alice actually sent to Bob
ring_members = txn["vin"][0]["key"]["key_offsets"]


print(f"Ring member offsets: {ring_members}")


Ring member offsets: [62550687, 5819092, 2889591, 297538, 364552, 29930, 15196, 56175, 40607, 22028, 12761, 29206, 463, 16754, 4656, 340]


The values listed above are index offsets. To create the actual index values a running sum is required where each new value is the sum of all prior values.

In [3]:
'''
Zero to Monero: Second Edition; Appendix A
'''


'''
To identify ring member output indexes this code performs the following:

  1. Calculates a running sum of output offsets
'''

# convert ring members to transaction output indexes
transaction_output_idx = []

# calculate a running sum of index offsets
transaction_output_idx = [sum(ring_members[0:x:1]) for x in range(0, len(ring_members) + 1)]

# the running sum double counts the first entry, remove one of the duplicated values
transaction_output_idx[1:]


print(f"Ring member output number: {transaction_output_idx}")

Ring member output number: [0, 62550687, 68369779, 71259370, 71556908, 71921460, 71951390, 71966586, 72022761, 72063368, 72085396, 72098157, 72127363, 72127826, 72144580, 72149236, 72149576]


Once Alice has the ring member decoy selections she needs two pieces of data from each decoy output, the output's stealth address, and the output's commitment. To know which output to use from a given transaction Alice needs to know which output in a transaction corresponds to a given output index. This is usually performed by wallet software and the author does not know of a `monerod` command that can retreive this data. Instead the author is using an external tool to identify the transaction index and output number that corresponds to a given output index. 

For example, the second decoy selected has output offset 5819092 which corresponds to output index 68369779. Output index 68369779 corresponds to the second output in the twenty-first transaction in block 281420. 

To clarify the following information is relevant for output offset 5819092:

* output index: 68369779
* block_id: 2814240
* transaction index: 20 (zero indexed)
* transaction hahs: 349752ec24178888cd47546aa1503c0227f8d909412f6d887495b428e0d77e16
* output number: 1 (zero indexed)

Decoy retrieval information Alice will use in creating a CLSAG signature is summarised in the following table.

| ring_member | output_idx | cumulative_output_idx | block    | txn_hash                                                         | output_num |
| ----------- | -----------| --------------------- | -------- | ---------------------------------------------------------------- | ---------- | 
| 00          | 62550687   | 62550687              | 02733614 | 7b98259b7370dd02e28c782a66662d4cd5b798bcae3f630b8658908cd943da3f | 02         |
| 01          | 5819092    | 68369779              | 02814240 | 349752ec24178888cd47546aa1503c0227f8d909412f6d887495b428e0d77e16 | 01         |
| 02          | 2889591    | 71259370              | 02857482 | 8e8eb586b9c3edc397ba24dc3c3ea1ac4c2a6a8d2d05848108a19a0400c70402 | 01         |
| 03          | 297538     | 71556908              | 02862292 | b6a756ba60acd4b9a3f2e29af0ade491913b09747c71965d42102b431ddf91e9 | 00         |
| 04          | 364552     | 71921460              | 02867940 | d54973b6b0ee7f78538aea9e7f824d20b810bfc50f00389dc31e224ce0999e79 | 339        |
| 05          | 29930      | 71951390              | 02868445 | 55d7dfe0c9bc694d0df53eb9eb7004b4faafe583ea6e51730064c54f6732124f | 01         |
| 06          | 15196      | 71966586              | 02868673 | 6332e87bb9f4aa00ca5f37c687861b7e84c57dc70039d90ca37d84f25bff3796 | 00         |
| 07          | 56175      | 72022761              | 02869576 | 3be7fc64abc7ce3cab6d5535b8227e32088ea2ddf35f70ba35c7839a1b604174 | 00         |
| 08          | 40607      | 72063368              | 02870350 | 6f1b95256ba3924e7f47312c0d502f54dc2145f95b0eec7a2ff011256ffb0c35 | 01         |
| 09          | 22028      | 72085396              | 02870727 | 9563ef7b9e4773bd3f2e02433be5404c13830037dc087df6214348a51368b847 | 01         |
| 10          | 12761      | 72098157              | 02870916 | 005847f72374b61ef8bc69d05adfe76e9cf2f379257c39f0d2b5ec32a133f88f | 00         |
| 11          | 29206      | 72127363              | 02871421 | 20d31deb7dfba410d7f3f6938a2be89e9b0911c6a86d53da24af77fe6a48949f | 01         |
| 12          | 463        | 72127826              | 02871427 | 7d03775f3555bc3c7b675d980991c9bdadd22ba3e2f6afed440d400c5e7830d5 | 01         |
| 13          | 16754      | 72144580              | 02871639 | 863206bb32e27a83b512b014035383438c32eb8802c4769661385e2144c7d157 | 10         |
| 14          | 4656       | 72149236              | 02871708 | 1f001f7ecdf8ef026c10b2fd7bf7b0a016657093d207abeda82f46ef0e5d16d0 | 00         |
| 15          | 340        | 72149576              | 02871712 | be71537a3857498b00c2458f8710d8dbc0de02018607857cc27b80afa242dc61 | 00         |

**Note: One of the ring members is Alice's real spend. In this example it happens to be the last ring member.**  

Columns 0:1 sourced from Alice's sent transaction  
Column  2   generated from column 1  
Columns 3:5 sourced from: https://xmrchain.net/

For this worked example decoy transaction outputs available from `monerod` were saved to json files. The two pieces of information Alice needs from each transaction (stealth address, commitment) will be read from those saved json files. Typically the data would be read directly from the blockchain.

In [4]:
'''
Not typically required
'''


'''
To retrieve ring member output data this code performs the following:

  1. Creates a list of dictionaries indicating the file to use and output to retrieve
  2. Retreives stealth addresses and commitments from json files
  3. Codes around an special case for mining coinbase decoys
'''

# specify list of ring members including Alice's real spend
ring_member_entries = [
    {"output_idx": 2,   "txn_file": "txn_7b98.json"}, # ring member 0
    {"output_idx": 1,   "txn_file": "txn_3497.json"}, # ring member 1
    {"output_idx": 1,   "txn_file": "txn_8e8e.json"}, # ring member 2
    {"output_idx": 0,   "txn_file": "txn_b6a7.json"}, # ring member 3
    {"output_idx": 339, "txn_file": "txn_d549.json"}, # ring member 4
    {"output_idx": 1,   "txn_file": "txn_55d7.json"}, # ring member 5
    {"output_idx": 0,   "txn_file": "txn_6332.json"}, # ring member 6
    {"output_idx": 0,   "txn_file": "txn_3be7.json"}, # ring member 7
    {"output_idx": 1,   "txn_file": "txn_6f1b.json"}, # ring member 8
    {"output_idx": 1,   "txn_file": "txn_9563.json"}, # ring member 9
    {"output_idx": 0,   "txn_file": "txn_0058.json"}, # ring member 10
    {"output_idx": 1,   "txn_file": "txn_20d3.json"}, # ring member 11
    {"output_idx": 1,   "txn_file": "txn_7d03.json"}, # ring member 12
    {"output_idx": 10,  "txn_file": "txn_8632.json"}, # ring member 13
    {"output_idx": 0,   "txn_file": "txn_1f00.json"}, # ring member 14
    {"output_idx": 0,   "txn_file": "txn_be71.json"}, # ring member 15 (Alice's owned output)
]

# iterate over list of ring members
for entry in ring_member_entries:
    output_idx = entry["output_idx"]
    txn_file = entry["txn_file"]
    with open(f"../../transactions/outbound/ring_members/{txn_file}", "r") as file:
        txn = json.load(file)
    
    # read stealth address
    entry["stealth_address"] = txn["vout"][output_idx]["target"]["tagged_key"]["key"]
    
    # mining coinbase decoys do not contain a commitment (the amount is in clear text)
    commitments = txn["rct_signatures"].get("outPk")
    
    # code around missing commitment for coinbase decoy
    entry["commitment"] = commitments[output_idx] if commitments else None


# print all ring member entries
for i, ring_member in enumerate(ring_member_entries):
    print(f"Ring member {i} -- stealth address: {ring_member['stealth_address']} -- commitment: {ring_member['commitment']}")

Ring member 0 -- stealth address: 728322fc4a25677c684f4dbf2474ff6dbb64d3714152b458f060dbbf47251ef6 -- commitment: f2edff407a6f2aada8366ba86c9c309815cacd6414ee99a33cc151be58e9931c
Ring member 1 -- stealth address: 117e7f5266b3e184de65f2859255ec53861f7bfb011044db3de1ec0deda51b9e -- commitment: dc471a2015f8ba953743402b8b449ef723e21d2ba2b582e4e66f313fb3639cae
Ring member 2 -- stealth address: 340c95f0d8784f0681788a214e49c098172c73dd84564a8d4f4caff406f57e86 -- commitment: d0a13129b298752171c874d75c7662ca466875c492ae0fb8ece397195517ffc3
Ring member 3 -- stealth address: 7cb815930bd9cc36915e9a3e5419ab57f7ec140095e85e229675852100cc5217 -- commitment: 96fd296e7173aa4403051774119f23388d9e048500a3fd5963fdd98e3b8a14a1
Ring member 4 -- stealth address: 1aceb49b860e5da34bfe9a759934ae35641b5907a4eeee6111605b21c0e42f6e -- commitment: None
Ring member 5 -- stealth address: faa164a08bd55ae8836e155cdda2000d95b6df4adb486d4d718862a7dbab0308 -- commitment: 9e578529c815ec3473f18e15788bf46dc66200e59432df8ab43

## 3. Recreating ring member commitment for mining output decoy

One of the decoys Alice happened to select was a mining coinbase output. The amounts for these outputs are in cleartext and the outputs do not record a commitment in the blockchain. Because a commitment is required for use in messaging signing (CLSAG), a commitment is created using the cleartext amount. Recall that a commitment is typically `yG + bH` where `y` in the blinding factor and `b` is the amount. The amount for a coinbase output is known, and since there is no corresponding blinding factor, `y` is simply set to the number 1. The corresponding commitment is `1*G + bH`. Coinbase commitments used as decoys in CLSAG are recreated using this formula by senders and validators. The commitment is recreated in this section. 

In [5]:
'''
Zero to Monero: Second Edition; Section 5.3
https://monero.stackexchange.com/questions/12264/how-to-do-mlsag-signatures-with-vin-of-type-0-miner/12271#12271
'''


'''
To create the coinbase decoy commitment this code performs the following:

  1. Recreates the curve point H using the base point G
  2. Reads the cleartext coinbase amount from transaction data
  3. Recreates the commitment using y = 1 (1*G + bH)
  4. Writes the commitment to the previously created list of ring members
'''

import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()


# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add



# recreate the curve point H
cofactor = int(8).to_bytes(32, byteorder="little")
G = scalar_mult_G(int(1).to_bytes(32, byteorder="little"))
H = keccak_256(G)
H = scalar_point_mult(cofactor, H)


# identify miner transaction data from previously created dictionary
miner_ring_member = ring_member_entries[4]
miner_txn_file = miner_ring_member["txn_file"]
miner_output_idx = miner_ring_member["output_idx"]

# read coinbase amount
with open(f"../../transactions/outbound/ring_members/{miner_txn_file}", "r") as file:
    miner_txn = json.load(file)
    amount = miner_txn["vout"][miner_output_idx]["amount"]


# calculate amount coefficient
b = int(amount).to_bytes(32, byteorder="little")

# calculate coinbase blinding factor (y = 1)
y = int(1).to_bytes(32, byteorder="little")

# calculate commitment inputs
yG = scalar_mult_G(y)
bH = scalar_point_mult(b, H)

# create "fake" commitment for coinbase transaction: C = 1*G + b*H; y=1, b=amount
commitment = hexlify(point_add(yG, bH)).decode()

# save commitment to appropriate dictionary in ring member list
ring_member_entries[4]["commitment"] = commitment


print(f"Recreated coinbase commitment: {commitment}")

Recreated coinbase commitment: fd2c1b18072013b5db10ec83e3646dfe563a5fed84c2e13dedab373bac9e1d72


In [6]:
# print all ring member entries
for i, ring_member in enumerate(ring_member_entries):
    print(f"Ring member {i} -- stealth address: {ring_member['stealth_address']} -- commitment: {ring_member['commitment']}")

Ring member 0 -- stealth address: 728322fc4a25677c684f4dbf2474ff6dbb64d3714152b458f060dbbf47251ef6 -- commitment: f2edff407a6f2aada8366ba86c9c309815cacd6414ee99a33cc151be58e9931c
Ring member 1 -- stealth address: 117e7f5266b3e184de65f2859255ec53861f7bfb011044db3de1ec0deda51b9e -- commitment: dc471a2015f8ba953743402b8b449ef723e21d2ba2b582e4e66f313fb3639cae
Ring member 2 -- stealth address: 340c95f0d8784f0681788a214e49c098172c73dd84564a8d4f4caff406f57e86 -- commitment: d0a13129b298752171c874d75c7662ca466875c492ae0fb8ece397195517ffc3
Ring member 3 -- stealth address: 7cb815930bd9cc36915e9a3e5419ab57f7ec140095e85e229675852100cc5217 -- commitment: 96fd296e7173aa4403051774119f23388d9e048500a3fd5963fdd98e3b8a14a1
Ring member 4 -- stealth address: 1aceb49b860e5da34bfe9a759934ae35641b5907a4eeee6111605b21c0e42f6e -- commitment: fd2c1b18072013b5db10ec83e3646dfe563a5fed84c2e13dedab373bac9e1d72
Ring member 5 -- stealth address: faa164a08bd55ae8836e155cdda2000d95b6df4adb486d4d718862a7dbab0308 -- com

## 4. Creating the message to be signed

Alice wants to sign a message that contains the transaction data she will submit to the blockchain. Most of the data she will sign has been generated in the previous two notebooks in this worked example. To create the message Alice will group all of the information together and then hash. Grouping the data together equivalent to turning all of the data into hex and then concatenating the data. Rather than performing that step in this worked example, the message data will be retrieved from the blockchain using `monerod print_tx <tx_hash> +hex`. That step was previously performed and the output saved to json. The information in that json will be retrived and then converted into a hashed message.

The saved transaction hex data from the blockchain contains the entire transaction, including CLSAG data which is not included in the message to be signed. The section below retrieves appropriate sections from transaction hex and creates a message. The message contains:

* header data: everything from `version` through `extra`
* output data: everything in the rct_signatures section
* bulletproof data: the rct_prunable section excluding integers indicating the number of bulletproofs, and number of L,R entries

In [7]:
'''
Zero to Monero: Second Edition; Section 6.2.2
'''


'''
To create the transaction message to be signed this code performs the following:

  1. Reads transaction hex previously saved to file
  2. Extracts the header section
  3. Extracts the rct_signatures section
  4. Extracts rct_prunable (excluding the integers previously mentioned, and CLSAG data)
  5. Hashes each of the three pieces of data separately
  6. Calculates an aggregate hash
'''

import json


# read data directly from the saved transaction
with open("../../transactions/outbound/txn_9e29_hex.json", "r") as file:
    txn = json.load(file)


# entire transaction converted to hex (monerod <txn_hash> +hex)
transaction_hex = txn["transaction_hex"]


# extract version through extra section of transaction
section_version_through_extra = transaction_hex[:400]

# hash this section of the transaction data
section_version_through_extra_hashed = keccak_256(unhexlify(section_version_through_extra))



# extract rct_signatures section of transaction
section_rct_signatures = transaction_hex[400:570]

# hash this section of the transaction data
section_rct_signatures_hashed = keccak_256(unhexlify(section_rct_signatures))



# extract rctsig_prunable section of transaction (exclude CLSAG)
section_rctsig_prunable_excluding_clsag = transaction_hex[572:956]
section_rctsig_prunable_excluding_clsag += transaction_hex[958:1406]
section_rctsig_prunable_excluding_clsag += transaction_hex[1408:1856]

# hash this section of the transaction data
section_rctsig_prunable_excluding_clsag_hashed = keccak_256(unhexlify(section_rctsig_prunable_excluding_clsag))


# joined the three hashes into one byte string and hash
message = keccak_256(section_version_through_extra_hashed + section_rct_signatures_hashed + section_rctsig_prunable_excluding_clsag_hashed)
message = hexlify(message).decode()


print(f"Transaction section -- top   : {section_version_through_extra}")
print(f"Transaction section -- middle: {section_rct_signatures}")
print(f"Transaction section -- bottom: {section_rctsig_prunable_excluding_clsag}")
print(f"Hashed message -- top   : {hexlify(section_version_through_extra_hashed).decode()}")
print(f"Hashed message -- middle: {hexlify(section_rct_signatures_hashed).decode()}")
print(f"Hashed message -- bottom: {hexlify(section_rctsig_prunable_excluding_clsag_hashed).decode()}")
print(f"Aggregate hashed message: {message}")

Transaction section -- top   : 0200010200109fe5e91dd495e302f7aeb001c2941288a016eae901dc76efb6039fbd028cac01d96396e401cf03f28201b024d4025631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd010200034e9451dd54125562c76cb59decc500b396fb2f6dc73f95876adaa6729c7db3f8bf000307ab8e6d3981d84b50f0d02ec45c2232ba239159db2f15adf44947535be13967c62c010286418eb6ae7e07927819e3ab0bfb9aede712ca02352522e9d0f081df3d873d020901462650609010339e
Transaction section -- middle: 068080d30ed9311c21fa3168e1d878feb0045449ee8d34848196e903ba8f9ff13d57402ee4e845aba06dbd2aee382f20c9ac8661573921477a4506f7eddde7ff228f05d8026525276dce1880b3d57e540836ccc8d1
Transaction section -- bottom: 759574339b52f6d9ca9cfea66b33095e43258b600fa84297287b02af6c69cd4a8c54f1e8d53900a1a054e47b04db85f81a87eff7ffe889c9a6d1c6efbd5c57f9e4ddde01092b17b9f7c606870869d1804e8b7713a49069531e532ff0a70a2b88c4942dce781872c2a510be6ee1bd5bdb23690c63e6fbca0cdf2efa51c7c8160245dd543cdbc4e7f4ef4e2c44b966f8c4d1a2550db938b6545a722ac6a763f70e2651e1146c01f8f

## 5. Creating CLSAG input data

Alice has accumulated most of the information she needs to sign the message with CLSAG. Using the data she has already accumulated Alice will created the final inputs required for the signature. To clarify the signing process for this worked example, the information Alice needs will be generated and then put into four separate lists. There will be 16 items in each list, one for each ring member.

Alice started with stealth addresses and commitments for each of the ring members. She will perform two additional actions:

1. Create a key image base for each of the stealth addresses using the `hash_to_point()` function. This is similar to the process Alice uses to create her own key image. Recall that the key image was calculated as `stealth_address_private_key * hash_to_point(stealth_address)`. Alice of course does not know the stealth address private key for ring member decoys, and will only calculate the second portion of that equation.  
**Note: this is the one place where the author uses outside functions. Monero uses  `hash_to_point` which maps public keys to new public keys where the private key for the new public key is unknown. The function is very similar to the `nacl.bindings.crypto_core_ed25519_from_uniform` function but with a different way of handling y-coordinates.** 
2. Create "fake" commitments to zero by taking the ring member commitments and then subtracting her own pseudo-commitment. The values created here are used in CLSAG but, because Alice does not know the amounts the ring member decoys represent, they are not true commitments to zero.

Alice will also create a key image for the commitment to zero. This is an auxiliary key that is stored in the transaction data in the CLSAG section as `D`. 

Finally, Alice will generate random numbers used in signing the message. Normally these are generated by wallet software and stored in the CLSAG section of the transaction. Because this is a worked example for a real transaction the random numbers will be read directly from the actual spend transaction. The list of random numbers will be modified to remove one value that will be created by the CLSAG signature.

In [8]:
# https://github.com/monero-project/mininero/blob/master/mininero.py#L238
# https://doc.libsodium.org/advanced/point-arithmetic#elligator-2-map

import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


q = 2**255 - 19
d = -121665 * pow(121666, -1, q) % q


def keccak_256(data) -> str:
    return keccak.new(digest_bits=256).update(data).digest()


def inv(x):
    return pow(x, -1, q)


def modp_inv(x):
    return pow(x, q - 2, q)


def sqroot(xx):
  x = pow(xx, (q+3)//8, q)
  if (x*x - xx) % q != 0:
    I = pow(2, (q-1)//4, q) 
    x = (x*I) % q
  if (x*x - xx) % q != 0: 
    print("no square root!")
  return x


def compress_point(x, y):
    parity = x & 1
    compressed_y = y.to_bytes(32, byteorder='little')
    if parity:
        msb = compressed_y[-1] | 0x80
        compressed_y = compressed_y[:-1] + bytes([msb])
    return hexlify(compressed_y).decode()


def mul_point_8(p):
    point_add = nacl.bindings.crypto_core_ed25519_add
    p2 = p
    for i in range(7):
        p2 = point_add(p, p2)
    return hexlify(p2).decode()


def hash_to_point(hexVal: bytes) -> str:
    u = int.from_bytes(keccak_256(hexVal), byteorder="little") % q
    A = 486662
    sqrtm1 = sqroot(-1)
    w = (2 * u * u + 1) % q
    xp = (w *  w - 2 * A * A * u * u) % q
    rx = pow(w * inv(xp), (q+3)//8, q)

    x = rx * rx * (w * w - 2 * A * A * u * u) % q
    y = (2 * u * u  + 1 - x) % q

    negative = False
    if (y != 0):
        y = (w + x) % q
        if (y != 0) :
            negative = True
        else :
            rx = rx * -1 * sqroot(-2 * A * (A + 2) ) % q
            negative = False
    else :
        rx = (rx * -1 * sqroot(2 * A * (A + 2) ) ) % q 
    if not negative:
        rx = (rx * u) % q
        z = (-2 * A * u * u)  % q
        sign = 0
    else:
        z = -1 * A
        x = x * sqrtm1 % q
        y = (w - x) % q 
        if (y != 0) :
            rx = rx * sqroot( -1 * sqrtm1 * A * (A + 2)) % q
        else :
            rx = rx * -1 * sqroot( sqrtm1 * A * (A + 2)) % q
        sign = 1
    #setsign
    if ( (rx % 2) != sign ):
        rx =  - (rx) % q 
    rz = (z + w) % q
    ry = (z - w) % q
    rx = rx * rz % q

    rz_inv = modp_inv(rz)
    x = rx * rz_inv % q
    y = ry * rz_inv % q
    p = compress_point(x,y)
    return mul_point_8(unhexlify(p))

In [9]:
'''
Zero to Monero: Second Edition; Section 3.6
'''


'''
To create CLSAG input data this code performs the following:

  1. Read data from the previously generated list of ring members
  2. Calculate key image bases for all ring members by using the hash_to_point() function
  3. Subtracts Alice's pseudo-commitment from each of the ring member commitments
'''


# define a function to subtract curve points
point_sub = nacl.bindings.crypto_core_ed25519_sub

# Alice's previously calculated pseudo-commitment
pseudo_commitment = "57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3"

# create empty lists for each of the four pieces of data
ring_member_stealth_addresses = []
ring_member_key_image_bases = []
ring_member_commitments = []
ring_member_commitments_zero = []

# iterate through the previously created ring members and save data to separate lists
for ring_member in ring_member_entries:
    stealth_address = ring_member["stealth_address"]
    commitment = ring_member["commitment"]
    ring_member_stealth_addresses.append(stealth_address)
    
    # calculate the key image base for each ring member
    ring_member_key_image_bases.append(hash_to_point(unhexlify(stealth_address)))
    ring_member_commitments.append(commitment)

    # calculate ring member commitment less Alice's pseudo-commitment for each ring member
    ring_member_commitments_zero.append(hexlify(point_sub(unhexlify(commitment), unhexlify(pseudo_commitment))).decode())


print(f"Ring member stealth addresses  : {ring_member_stealth_addresses}")
print(f"Ring member key image bases    : {ring_member_key_image_bases}")
print(f"Ring member commitments        : {ring_member_commitments}")
print(f"Ring member commitments to zero: {ring_member_commitments_zero}")

Ring member stealth addresses  : ['728322fc4a25677c684f4dbf2474ff6dbb64d3714152b458f060dbbf47251ef6', '117e7f5266b3e184de65f2859255ec53861f7bfb011044db3de1ec0deda51b9e', '340c95f0d8784f0681788a214e49c098172c73dd84564a8d4f4caff406f57e86', '7cb815930bd9cc36915e9a3e5419ab57f7ec140095e85e229675852100cc5217', '1aceb49b860e5da34bfe9a759934ae35641b5907a4eeee6111605b21c0e42f6e', 'faa164a08bd55ae8836e155cdda2000d95b6df4adb486d4d718862a7dbab0308', '02e9350901316954b71d457af7eb06a8db5a96071b5be1c7cad8f0252bd264c9', 'b6ec675aa98eac922feb4241baccb66719c9cbde6d0d066f725c3cb6ff2893c3', '6dc2dd4b709feb2e7c186258cbe72479f311406187bbcf0c3bd957c5548f24fd', 'ca71c67bb58f8dc9f0500872d2b3792f5f8cce16f53a6f9738b9f391ef6c3a50', '7de760ff4510c632dbb9f19cc94e8953b209840199c8f69b5e9e7278d25f9c4c', 'c18433de6b67e75bbe69b7076f6d8d3e3ba5015a15b8d9b1ed110e2875511c82', '13d37a3d9dda81af6e30777c9be8c2ae93e1f1497a07e043465fd83b91de368b', 'a829cbcc2bb6d075c12e866d3fc8e87c50d92ab131abbc18f5b744165a46333a', '85601b8eaf201

In [10]:
'''
Zero to Monero: Second Edition; Section 3.6
'''


'''
To create the commitment to zero key image this code performs the following:

  1. Calculates the inverse of the cofactor 8 = 1/8
  2. Multiplies the commitment to zero private key by 1/8
  3. Multiplies the private key in step 2 by the key image base [hash_to_point(alice_stealth_address)]
'''

# define a function to invert a scalar modulo the prime order0
scalar_invert = nacl.bindings.crypto_core_ed25519_scalar_invert

# define a function to multiply two ed25519 scalars
scalar_scalar_mult = nacl.bindings.crypto_core_ed25519_scalar_mul

# convert the integer 8 to 32 bytes in little-endian order
cofactor = int(8).to_bytes(32, byteorder="little")

# calculates 1/8
cofactor_inverse = scalar_invert(cofactor)

# multiplies the commitment to zero private key by the key image base
commitment_to_zero_key_image = scalar_point_mult(commitment_to_zero_private_key, unhexlify(ring_member_key_image_bases[-1]))

# multiplies the above result by 1/8
commitment_to_zero_key_image = hexlify(scalar_point_mult(cofactor_inverse, commitment_to_zero_key_image)).decode()



print(f"Commitment to zero key image (D): {commitment_to_zero_key_image}")

Commitment to zero key image (D): dab0bd2d0c1151a2478f70ed6fe9d881b63635ff50f7329e79be842599497619


In [11]:
'''
Not normally required
'''


'''
To identify random numbers used in CLSAG signing this code performs the following:

  1. Reads the random numbers from real transaction data
  2. Removes the last entry that will be calculated by CLSAG
'''


with open("../../transactions/outbound/txn_9e29.json", "r") as file:
    txn = json.load(file)


# read random numbers from real transaction data
random_numbers = txn["rctsig_prunable"]["CLSAGs"][0]["s"]

# remove the last entry in the list that will be recreated
del random_numbers[-1]


print(f"Random numbers used in CLSAG (s): {random_numbers}")

Random numbers used in CLSAG (s): ['d1daaaf3d6405cab232858da4df07a2ce1349d75952cc1b406d2f959cb66ab06', 'ac7545c4ea773688bceb0cd01df42ed26825bf5b44df7b7f511dd3352eb5490a', '0b72dcd5dbfb9aa79564054d11e616a41f8e23e7875f92a05b7d18d189fda40b', '76a1b0c683ab983cdac0ca7df2fbb4d509dc95937c6151a5ab1d7468539d7c01', '2c0c1068499384526e990de6de30a2ebc7dc563f29c0de9e5dbd03bf87144902', 'f48aab77cf8112ad762c90f0ddfec07eeb99b7cbc240500e8af905d7aa0efa06', '1adcc2075753aae9f287c36da9db269d227183bd359a3e908ecadd74d6021801', '0f270fabeff6e51f320514292ce730330a0d2ca1e743eaeb3593e84e8e315d09', 'efa182f646cd1202f8d46e23b214ba640adf9ffa243658b312646bc57152a300', '511f0a42660f52db914db3232b6d02acae8fcf76c6dee4734ddb4c11e60ae407', 'd64f184fee06c09e7027467dc1a6e42fef09d1dfb9550b5e69419c7b7f005303', '84dcbd90d134433e6e92c740550de450d3755cc8ec86db06150c3153cd4fc90b', 'c692d275e67cdd9ec5dd4243bc5b84e755ba82a7d2f72323ab6f2a05e5bf3a00', 'adbf3643bfe99fe34fd66d04e4b763cd81972a08fdae6f937c9ec9989096e104', '37a8ce33e0dd

## 6.Create a valid CLSAG signature

This section covers the calculations in section 3.6. of Zero to Monero as well as using some information in chapter 6. Prior to CLSAG, another signing mechanism, MLSAG was used. The book Zero to Monero covers MLSAG in more detail.

In this worked example Alice's spend is the last ring member. Ring members 0 through 14 are decoys. This simplifies the below calculations because Alice can calculate her expected CLSAG output first and then simply loop through the list of decoy ring members starting at 0 and ending at 14. If her true spend was at another position, for example 7, then she would create her expected CLSAG output and then loop through 8 to 15 and then 0 to 6.

In [12]:
'''
Zero to Monero: Second Edition; Section 3.6
'''


'''
To create the CLSAG signature this code performs the following:

  1. Separately concatenates a ring member stealth addresses and commitments
  2. Creates stealth address and commitment signatures
  3. Create an aggregate private key using stealth address and commitment to zero private keys
  4. Calculates Alice's expected CLSAG output
  5. Iterates through all of the decoy ring members to create CLSAG outputs
  6. Uses the last CLSAG output from step 5 to create the final "random" number
'''

# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(point):
    point_padded_to_64_bytes = point + (64 - len(point)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


# define a function to multiply a scalar by a scalar
scalar_scalar_mult = nacl.bindings.crypto_core_ed25519_scalar_mul

# define a function to multiply a curve point by a scalar
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# define a function to multiply the ed25519 generator point G by a scalar (eg: integer)
point_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to add two scalars (eg: two integers)
scalar_add = nacl.bindings.crypto_core_ed25519_scalar_add

# define a function to subtract two scalars (eg: two integers)
scalar_sub = nacl.bindings.crypto_core_ed25519_scalar_sub


# define Alice's previously calculated key image
alice_key_image = "5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01"

# define Alice's previously calculated stealth address private key
stealth_address_private_key = unhexlify("9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06")

# define a random number to use in CLSAG (normally generated by wallet software)
a = "ec3e2ea6127b4aa7f811dac2f388d39ccf2ec7b6108a7f879b406d6023b11a06"


# define three different hex strings to prepend to hash data
domain0 = hexlify(b"CLSAG_agg_0").decode().ljust(64,"0")
domain1 = hexlify(b"CLSAG_agg_1").decode().ljust(64,"0")
domain_round = hexlify(b"CLSAG_round").decode().ljust(64,"0")

# concatenate all ring member stealth addresses (including Alice's)
ring_member_stealth_addresses_joined = "".join(ring_member_stealth_addresses)

# concatenate all ring member commitments (including Alice's)
ring_member_commitments_joined = "".join(ring_member_commitments)


# generate the signatures (only the first "domain" string is different)
public_key_signature = keccak_256(unhexlify(domain0 + ring_member_stealth_addresses_joined + ring_member_commitments_joined + alice_key_image + commitment_to_zero_key_image + pseudo_commitment))
public_key_signature = pad_and_reduce(public_key_signature)

# generate the signatures (only the first "domain" string is different)
commitment_signature = keccak_256(unhexlify(domain1 + ring_member_stealth_addresses_joined + ring_member_commitments_joined + alice_key_image + commitment_to_zero_key_image + pseudo_commitment))
commitment_signature = pad_and_reduce(commitment_signature)


# Section 3.6: step 3; create aggregate private key (Note this is the lowercase "w" and the end of step 3)
w_stealth_address = scalar_scalar_mult(stealth_address_private_key, public_key_signature)
w_commitment_to_zero = scalar_scalar_mult(commitment_to_zero_private_key, commitment_signature)
w = scalar_add(w_stealth_address, w_commitment_to_zero)

# Section 3.6: step 4; create "c" using Alice's information
aG = hexlify(point_mult_G(unhexlify(a))).decode()
aH = hexlify(scalar_point_mult(unhexlify(a), unhexlify(ring_member_key_image_bases[-1]))).decode()
c = keccak_256(unhexlify(domain_round + ring_member_stealth_addresses_joined + ring_member_commitments_joined + pseudo_commitment + message + aG + aH))
c = hexlify(pad_and_reduce(c)).decode()

# in this worked example Alice's output is the last member of the ring, so her "c" becomes "c1"
c1 = c


# Section 3.6: step 5
for i, random_number in enumerate(random_numbers):

    # left portion of formula [rG + cW]
    rG = point_mult_G(unhexlify(random_number))
    W_stealth_address = scalar_point_mult(public_key_signature, unhexlify(ring_member_stealth_addresses[i]))
    W_commitment_zero = scalar_point_mult(commitment_signature, unhexlify(ring_member_commitments_zero[i]))
    W = point_add(W_stealth_address, W_commitment_zero)
    cW = scalar_point_mult(unhexlify(c), W)
    left_formula_block = hexlify(point_add(rG, cW)).decode()

    # right portion of formula [rH + cW~]
    rH = scalar_point_mult(unhexlify(random_number), unhexlify(ring_member_key_image_bases[i]))
    W_key_image = scalar_point_mult(public_key_signature, unhexlify(alice_key_image))
    W_commitment_zero_key_image = scalar_point_mult(commitment_signature, unhexlify(commitment_to_zero_key_image))
    W_commitment_zero_key_image = scalar_point_mult(cofactor, W_commitment_zero_key_image)
    W = point_add(W_key_image, W_commitment_zero_key_image)
    cW = scalar_point_mult(unhexlify(c), W)
    right_formula_block = hexlify(point_add(rH, cW)).decode()

    str_hash = domain_round + ring_member_stealth_addresses_joined + ring_member_commitments_joined + pseudo_commitment + message + left_formula_block + right_formula_block

    c = keccak_256(unhexlify(str_hash))
    c = hexlify(pad_and_reduce(c)).decode()


# Section 3.6: step 5; calculate cw then subtract cw from a (a - cw), this becomes the final "random number"
cw = scalar_scalar_mult(unhexlify(c), w)
r_pi = hexlify(scalar_sub(unhexlify(a), cw)).decode()
random_numbers.append(r_pi)


print(f"CLSAG completed list of scalars (s)   : {random_numbers}")
print(f"CLSAG first round input (c1)          : {c1}")
print(f"CLSAG commitment to zero key image (D): {commitment_to_zero_key_image}")


CLSAG completed list of scalars (s)   : ['d1daaaf3d6405cab232858da4df07a2ce1349d75952cc1b406d2f959cb66ab06', 'ac7545c4ea773688bceb0cd01df42ed26825bf5b44df7b7f511dd3352eb5490a', '0b72dcd5dbfb9aa79564054d11e616a41f8e23e7875f92a05b7d18d189fda40b', '76a1b0c683ab983cdac0ca7df2fbb4d509dc95937c6151a5ab1d7468539d7c01', '2c0c1068499384526e990de6de30a2ebc7dc563f29c0de9e5dbd03bf87144902', 'f48aab77cf8112ad762c90f0ddfec07eeb99b7cbc240500e8af905d7aa0efa06', '1adcc2075753aae9f287c36da9db269d227183bd359a3e908ecadd74d6021801', '0f270fabeff6e51f320514292ce730330a0d2ca1e743eaeb3593e84e8e315d09', 'efa182f646cd1202f8d46e23b214ba640adf9ffa243658b312646bc57152a300', '511f0a42660f52db914db3232b6d02acae8fcf76c6dee4734ddb4c11e60ae407', 'd64f184fee06c09e7027467dc1a6e42fef09d1dfb9550b5e69419c7b7f005303', '84dcbd90d134433e6e92c740550de450d3755cc8ec86db06150c3153cd4fc90b', 'c692d275e67cdd9ec5dd4243bc5b84e755ba82a7d2f72323ab6f2a05e5bf3a00', 'adbf3643bfe99fe34fd66d04e4b763cd81972a08fdae6f937c9ec9989096e104', '37a8ce

## Optional: Retrieving "a"

The random value `a` used to create a CLSAG signature is not saved in transaction data or a wallet. In this worked example it was recreated using the following code. The code performs CLSAG validation using transaction data and then "back into" `a` using an intermediate value of `c`. This is only useful for the worked example.

In [13]:

scalar_add = nacl.bindings.crypto_core_ed25519_scalar_add
scalar_scalar_mult = nacl.bindings.crypto_core_ed25519_scalar_mul

c = unhexlify("956b70646a3dd6dfb48990fa7cf79d084fd80c5528bf8b3a472a1c6a716dff07")
r_pi = unhexlify("b381bae0a2a28df0aff6f674e4aadba267e7dd0d2bebcfc0b503e99e68d80b0c")

w_stealth_address = scalar_scalar_mult(public_key_signature, stealth_address_private_key)
w_commitment_to_zero = scalar_scalar_mult(commitment_signature, commitment_to_zero_private_key)
w = scalar_add(w_stealth_address, w_commitment_to_zero)
cw = scalar_scalar_mult(c, w)
a = scalar_add(r_pi, cw)

print(f"Retrieved (a) from CLSAG validation: {hexlify(a).decode()}")

Retrieved (a) from CLSAG validation: ec3e2ea6127b4aa7f811dac2f388d39ccf2ec7b6108a7f879b406d6023b11a06


### Summary
**What we start with:**
* spend key image: `5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01`
* spend private key: `9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06`
* spend pseudo-commitment: `57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3`
* received input blinding factor: `67f2cd5597666bef76bf1dd4a40d201ea4b4013848332b68bbabcbbe8bf9a408`
* pseudo-commitment blinding factor: `3d5a7e3341949f499dde51deeac45cbf17b8d2ad073c7a28e0a0134ed46eaa08`

**What we end up with:**
* commitment to zero private key: `176c457f7035defdaf7dc3989842a2738cfc2e8a40f7b03fdb0ab870b78afa0f`
* ring members: too long to list (16 entries), Alice is at position 15 (zero indexing)
* CLSAG random number inputs (s): too long to list (16 entries)
* CLSAG verification input (c1): `56f5c495ddc41375fcfe7762ba91850cd6bfa55117e397f6eff11ca41249ec07`
* CLSAG commitment to zero key image (D): `dab0bd2d0c1151a2478f70ed6fe9d881b63635ff50f7329e79be842599497619`